In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Dem0").getOrCreate()



24/08/16 13:25:41 WARN Utils: Your hostname, manu-pc resolves to a loopback address: 127.0.1.1; using 192.168.157.41 instead (on interface wlp58s0)
24/08/16 13:25:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/16 13:25:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/16 13:25:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## 1. How can you create a DataFrame a) using existing RDD, and b) from a CSV file?

In [2]:
rdd = spark.sparkContext.parallelize([
    (1, "Alice", 29),
    (2, "Bob", 31),
    (3, "Cathy", 24)
])
rdd.collect()

[(1, 'Alice', 29), (2, 'Bob', 31), (3, 'Cathy', 24)]

In [3]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

schema = StructType(fields=[
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("Age", IntegerType(), True),

])

df = rdd.toDF(schema=schema)
df.show()

+---+-----+---+
| id| name|Age|
+---+-----+---+
|  1|Alice| 29|
|  2|  Bob| 31|
|  3|Cathy| 24|
+---+-----+---+



In [4]:
import os
print(os.getcwd())
os.chdir(r"/media/manu/sec_storage/project_2024_2u/projects24/pyspark_prac/")
print(os.getcwd())


/media/manu/sec_storage/project_2024_2u/projects24/pyspark_prac/Notebook/inter_prep
/media/manu/sec_storage/project_2024_2u/projects24/pyspark_prac


In [15]:
file_path = r"/media/manu/sec_storage/project_2024_2u/projects24/pyspark_prac/record.csv"
schema = StructType(fields=[
    StructField("f_name", StringType(), True),
    StructField("m_name", StringType(), True),
    StructField("l_name", StringType(), True),
    StructField("id", IntegerType(), True),
    StructField("gender", StringType(), True),

])
df = spark.read.csv(file_path,schema=schema)

In [16]:
df.printSchema()

root
 |-- f_name: string (nullable = true)
 |-- m_name: string (nullable = true)
 |-- l_name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)



## 2. What are the different ways to handle row duplication in a PySpark DataFrame?

In [19]:
data = [("James", "Sales", 3000), \

    ("Michael", "Sales", 4600), \

    ("Robert", "Sales", 4100), \

    ("Maria", "Finance", 3000), \

    ("James", "Sales", 3000), \

    ("Scott", "Finance", 3300), \

    ("Jen", "Finance", 3900), \

    ("Jeff", "Marketing", 3000), \

    ("Kumar", "Marketing", 2000), \

    ("Saif", "Sales", 4100) \

  ]

df = spark.createDataFrame(data,['name',"dept",'salary'])
df.show()

+-------+---------+------+
|   name|     dept|salary|
+-------+---------+------+
|  James|    Sales|  3000|
|Michael|    Sales|  4600|
| Robert|    Sales|  4100|
|  Maria|  Finance|  3000|
|  James|    Sales|  3000|
|  Scott|  Finance|  3300|
|    Jen|  Finance|  3900|
|   Jeff|Marketing|  3000|
|  Kumar|Marketing|  2000|
|   Saif|    Sales|  4100|
+-------+---------+------+



In [21]:
df.dropDuplicates().show()

+-------+---------+------+
|   name|     dept|salary|
+-------+---------+------+
|  James|    Sales|  3000|
|Michael|    Sales|  4600|
| Robert|    Sales|  4100|
|  Maria|  Finance|  3000|
|  Scott|  Finance|  3300|
|    Jen|  Finance|  3900|
|   Jeff|Marketing|  3000|
|  Kumar|Marketing|  2000|
|   Saif|    Sales|  4100|
+-------+---------+------+



In [22]:
df.distinct().show()

+-------+---------+------+
|   name|     dept|salary|
+-------+---------+------+
|  James|    Sales|  3000|
|Michael|    Sales|  4600|
| Robert|    Sales|  4100|
|  Maria|  Finance|  3000|
|  Scott|  Finance|  3300|
|    Jen|  Finance|  3900|
|   Jeff|Marketing|  3000|
|  Kumar|Marketing|  2000|
|   Saif|    Sales|  4100|
+-------+---------+------+



In [23]:
df.dropDuplicates(["dept"]).show()

+-----+---------+------+
| name|     dept|salary|
+-----+---------+------+
|Maria|  Finance|  3000|
| Jeff|Marketing|  3000|
|James|    Sales|  3000|
+-----+---------+------+



##  Explain PySpark UDF with the help of an example.

In [16]:
from pyspark.sql.functions import udf

column = ["Seqno","Name"]

data = [("1", "john jones"),

    ("2", "tracey smith"),

    ("3", "amy sanders")]

df = spark.createDataFrame(data,column)
df.show()

+-----+------------+
|Seqno|        Name|
+-----+------------+
|    1|  john jones|
|    2|tracey smith|
|    3| amy sanders|
+-----+------------+



In [3]:
from pyspark.sql.functions import col, split  

df.withColumn("f_name", split(col("Name"), " ").getItem(0)) \
    .withColumn("l_name", split(col("Name"), " ").getItem(1)).show()

+-----+------------+------+-------+
|Seqno|        Name|f_name| l_name|
+-----+------------+------+-------+
|    1|  john jones|  john|  jones|
|    2|tracey smith|tracey|  smith|
|    3| amy sanders|   amy|sanders|
+-----+------------+------+-------+



In [17]:
from pyspark.sql.functions import col
from pyspark.sql.types import ArrayType,StringType
def seperate_name(name):
    names = []
    for word in name.split(" "):
        names.append(word)
    return names

# seperate_name_udf = udf(seperate_name,ArrayType(StringType()))
seperate_name_udf = udf(seperate_name)


df = df.withColumn("name", seperate_name_udf(col("Name")))
df.show()

+-----+---------------+
|Seqno|           name|
+-----+---------------+
|    1|  [john, jones]|
|    2|[tracey, smith]|
|    3| [amy, sanders]|
+-----+---------------+



In [18]:
df.printSchema()

root
 |-- Seqno: string (nullable = true)
 |-- name: string (nullable = true)



In [20]:
df.withColumn('name', split(col("name"),",")).show()


+-----+------------------+
|Seqno|              name|
+-----+------------------+
|    1|  [[john,  jones]]|
|    2|[[tracey,  smith]]|
|    3| [[amy,  sanders]]|
+-----+------------------+



In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("example") \
    .getOrCreate()

# Example DataFrame
data = [(1, "john jones"), (2, "tracey smith"), (3, "amy sanders")]
df = spark.createDataFrame(data, ["Seqno", "Name"])

# Define the UDF
def seperate_name(name):
    return name.split(" ")

# Register the UDF
seperate_name_udf = udf(seperate_name, ArrayType(StringType()))

# Apply the UDF to create a new column
df = df.withColumn("NameArray", seperate_name_udf(col("Name")))

# Show the DataFrame
df.show(truncate=False)

# Optional: Split the NameArray into separate columns
from pyspark.sql.functions import explode

# Convert the array to separate rows
df_exploded = df.withColumn("Name", explode(col("NameArray")))

# Show the exploded DataFrame
df_exploded.show()

# Stop the SparkSession when done
spark.stop()


24/08/16 13:17:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----+------------+---------------+
|Seqno|Name        |NameArray      |
+-----+------------+---------------+
|1    |john jones  |[john, jones]  |
|2    |tracey smith|[tracey, smith]|
|3    |amy sanders |[amy, sanders] |
+-----+------------+---------------+

+-----+-------+---------------+
|Seqno|   Name|      NameArray|
+-----+-------+---------------+
|    1|   john|  [john, jones]|
|    1|  jones|  [john, jones]|
|    2| tracey|[tracey, smith]|
|    2|  smith|[tracey, smith]|
|    3|    amy| [amy, sanders]|
|    3|sanders| [amy, sanders]|
+-----+-------+---------------+



## Discuss the map() transformation in PySpark DataFrame with the help of an example.

In [21]:
records = ["Project","Gutenberg’s","Alice’s","Adventures",

"in","Wonderland","Project","Gutenberg’s","Adventures",

"in","Wonderland","Project","Gutenberg’s"]

In [23]:
rdd = spark.sparkContext.parallelize(records)

In [26]:
rdd2 = rdd.map(lambda x: (x,1))
rdd2.collect()

[('Project', 1),
 ('Gutenberg’s', 1),
 ('Alice’s', 1),
 ('Adventures', 1),
 ('in', 1),
 ('Wonderland', 1),
 ('Project', 1),
 ('Gutenberg’s', 1),
 ('Adventures', 1),
 ('in', 1),
 ('Wonderland', 1),
 ('Project', 1),
 ('Gutenberg’s', 1)]

## Finding the rows which is in df_a but not in df_b

In [27]:
# Example DataFrames
data_a = [(1, "Alice"), (2, "Bob"), (3, "Cathy")]
data_b = [(2, "Bob"), (4, "David")]

df_a = spark.createDataFrame(data_a, ["id", "name"])
df_b = spark.createDataFrame(data_b, ["id", "name"])


In [28]:
df_diff = df_a.exceptAll(df_b)
df_diff.show()


+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  3|Cathy|
+---+-----+



In [29]:
from pyspark.sql.functions import col

# Perform LEFT JOIN
df_joined = df_a.join(df_b, on=["id", "name"], how="left_anti")

df_joined.show()


+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  3|Cathy|
+---+-----+



## What is meant by PySpark MapType? How can you create a MapType using StructType?


In [32]:
from pyspark.sql.types import MapType,StructType, StructField, StringType

schema = StructType(fields=[
    StructField("name", StringType(), True),
    StructField("properties", MapType(StringType(), StringType(), True))

])

In [34]:
dataDictionary = [

        ('James',{'hair':'black','eye':'brown'}),

        ('Michael',{'hair':'brown','eye':None}),

        ('Robert',{'hair':'red','eye':'black'}),

        ('Washington',{'hair':'grey','eye':'grey'}),

        ('Jefferson',{'hair':'brown','eye':''})

        ]

spark.createDataFrame(dataDictionary, schema).show(truncate=False)

+----------+-----------------------------+
|name      |properties                   |
+----------+-----------------------------+
|James     |{eye -> brown, hair -> black}|
|Michael   |{eye -> NULL, hair -> brown} |
|Robert    |{eye -> black, hair -> red}  |
|Washington|{eye -> grey, hair -> grey}  |
|Jefferson |{eye -> , hair -> brown}     |
+----------+-----------------------------+



## What is the function of PySpark's pivot() method?

In [44]:
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \

      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \

      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \

      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]

columns= ["Product","Amount","Country"]



In [45]:
df = spark.createDataFrame(data,columns)
df.printSchema()

root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)



In [46]:
pivot_df = df.groupBy("Product").pivot("Country").sum()
pivot_df.show()

+-------+------+-----+------+----+
|Product|Canada|China|Mexico| USA|
+-------+------+-----+------+----+
| Orange|  NULL| 4000|  NULL|4000|
|  Beans|  NULL| 1500|  2000|1600|
| Banana|  2000|  400|  NULL|1000|
|Carrots|  2000| 1200|  NULL|1500|
+-------+------+-----+------+----+



# Brodcast

Broadcast variables in PySpark are read-only shared variables that are stored and accessible on all nodes in a cluster so that processes may access or use them. Instead of sending this information with each job, PySpark uses efficient broadcast algorithms to distribute broadcast variables among workers, lowering communication costs.

The broadcast(v) function of the SparkContext class is used to generate a PySpark Broadcast. This method accepts the broadcast parameter v.

In [43]:
states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),

    ("Michael","Rose","USA","NY"),

    ("Robert","Williams","USA","CA"),

    ("Maria","Jones","USA","FL")

  ]

rdd = spark.sparkContext.parallelize(data)
def state_convert(code):
    return broadcastStates.value(code)

result = rdd.map(lambda x: (x[0], x[1], x[2], state_convert(x[3])) ).collect()

for item in result:
    print(result)

24/08/16 14:15:49 ERROR Executor: Exception in task 3.0 in stage 57.0 (TID 155)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/manu/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/manu/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/manu/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/manu/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_33279/634084980.py", line 18, in <lambda>
  File "/tmp/ipykernel_33279/634084980.py", line 16, in state_convert
TypeError: 'dict' object is not callable

	at org.apa

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 57.0 failed 1 times, most recent failure: Lost task 3.0 in stage 57.0 (TID 155) (192.168.157.41 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/manu/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/manu/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/manu/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/manu/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wra

In [47]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("example") \
    .getOrCreate()

# Broadcast variable
states = {"NY": "New York", "CA": "California", "FL": "Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

# Data
data = [("James", "Smith", "USA", "CA"),
        ("Michael", "Rose", "USA", "NY"),
        ("Robert", "Williams", "USA", "CA"),
        ("Maria", "Jones", "USA", "FL")]

# Create an RDD
rdd = spark.sparkContext.parallelize(data)

# Function to convert state code to state name
def state_convert(code):
    return broadcastStates.value.get(code, "Unknown")  # Use get() to handle missing keys

# Transform the RDD
result = rdd.map(lambda x: (x[0], x[1], x[2], state_convert(x[3]))).collect()

# Print the results
for item in result:
    print(item)

# Stop the SparkSession when done
spark.stop()



24/08/16 14:16:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


('James', 'Smith', 'USA', 'California')
('Michael', 'Rose', 'USA', 'New York')
('Robert', 'Williams', 'USA', 'California')
('Maria', 'Jones', 'USA', 'Florida')


In [49]:
states = {"NY":"New York", "CA":"California", "FL":"Florida"}

broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),

    ("Michael","Rose","USA","NY"),

    ("Robert","William","USA","CA"),

    ("Maria","Jones","USA","FL")

  ]

columns = ["firstname","lastname","country","state"]

df = spark.createDataFrame(data = data, schema = columns)

df.printSchema()

df.show(truncate=False)

def state_convert(code):

    return broadcastStates.value[code]

res = df.rdd.map(lambda a: (a[0],a[1],a[2],state_convert(a[3]))).toDF(column)

res.show(truncate=False)

AttributeError: 'NoneType' object has no attribute 'broadcast'

In [50]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("example") \
    .getOrCreate()

# Broadcast variable
states = {"NY": "New York", "CA": "California", "FL": "Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

# Data
data = [("James", "Smith", "USA", "CA"),
        ("Michael", "Rose", "USA", "NY"),
        ("Robert", "William", "USA", "CA"),
        ("Maria", "Jones", "USA", "FL")]

# Define schema
columns = ["firstname", "lastname", "country", "state"]

# Create DataFrame
df = spark.createDataFrame(data, schema=columns)

# Print schema and show DataFrame
df.printSchema()
df.show(truncate=False)

# Function to convert state code to state name
def state_convert(code):
    return broadcastStates.value.get(code, "Unknown")  # Use get() to handle missing keys

# Convert the state codes using RDD transformation and create a new DataFrame
res_rdd = df.rdd.map(lambda a: (a[0], a[1], a[2], state_convert(a[3])))
res_df = res_rdd.toDF(columns)

# Show the transformed DataFrame
res_df.show(truncate=False)

# Stop the SparkSession when done
spark.stop()


24/08/16 14:21:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)



+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|James    |Smith   |USA    |CA   |
|Michael  |Rose    |USA    |NY   |
|Robert   |William |USA    |CA   |
|Maria    |Jones   |USA    |FL   |
+---------+--------+-------+-----+

+---------+--------+-------+----------+
|firstname|lastname|country|state     |
+---------+--------+-------+----------+
|James    |Smith   |USA    |California|
|Michael  |Rose    |USA    |New York  |
|Robert   |William |USA    |California|
|Maria    |Jones   |USA    |Florida   |
+---------+--------+-------+----------+

